In [ ]:
# pip install tensorflow-hub
# pip install tensorflow-datasets
# done at terminal - just know we need these python modules

In [ ]:
# all the pre-requirements

import os
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
# weird that they don't call hub as tfhub to keep consistency, what gives?
import tensorflow_datasets as tfds

print('Version: ', tf.__version__)
print('Eager mode: ', tf.executing_eagerly())
print('Hub Version: ', hub.__version__)
print ('GPU is', 'available' if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

In [ ]:
# download the dataset, this is that same as the other imdb dataset we use but directly from tfds

print ('Loading imdb dataset from tfds ... may take a while')
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews",
    split=('train[:60%]', 'train[:60%]', 'test'),
    as_supervised=True)
print ('done downloading dataset')

In [ ]:
# explorint dataset

train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

In [ ]:
# also print labels

train_labels_batch
# as expected, it's 0 = neg review, 1 = positive review

In [ ]:
# this is where this example differs, we are now using a prebuilt model

print ('starting .. might take a min')
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])
print ('done!')

In [ ]:
# building the rest/full model

# notice how all the pre-processing is not here ...

model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

In [ ]:
# compile model

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

print('done compiling.')

In [ ]:
# training the model

history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1)

print ('done fitting')

In [ ]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.5f" % (name, value))

In [ ]:
# That's it .. see how small the number of steps in this example becaues we used pre-built models.